In [1]:
from adam_impact_study.population import generate_population, PopulationConfig
from adam_core.orbits import Orbits
from adam_core.time import Timestamp
import glob
import os

In [2]:
# Get default population config
population_config = PopulationConfig.default()

# Set diameters (in km)
diameters = [100, 500, 1000]

# Directory containing raw result files
raw_results_dir = "results_2025_10k_parquet_raw"

# Get all raw result files
result_files = glob.glob(f"{raw_results_dir}/*.parquet")

for result_file in result_files:
    # Load orbits from result file
    orbits = Orbits.from_parquet(result_file)
    
    # Extract dates from filename
    filename = os.path.basename(result_file)
    dates = filename.split("_")[1:3]  # Gets the start and end dates
    start_date = dates[0]
    end_date = dates[1].replace(".parquet", "")
    
    # Create impact dates timestamp
    impact_dates = orbits.coordinates.time
    
    # Generate population
    population = generate_population(
        orbits=orbits,
        impact_dates=impact_dates,
        population_config=population_config,
        diameters=diameters,
        albedo_distribution="rayleigh"
        seed=612
    )
    
    # Save population to parquet file
    output_filename = f"4.3.1_{start_date[:4]}-{end_date[:4]}_params.parquet"
    population.to_parquet(output_filename)